## Data Preparation/ for Model Training
### Steps:
- First we need to create `Input` folder with 2 subfolders: `BSI_data`, `Public_data`and paste the corresponding files in each folder.

    1. **BSI_data folder:**

        > `one_off_data`

        > `BSI_CREDITREPORTING`

        > `MLW_LOGINS`

    2. **Public_data folder:**
        > `Census_Demographic`
        
        > `HPI`
        
        > `Interest_Rate`

        > `Zip_FIPS_Mapping`

Note: All dataset file names should be name as same.

#### Install the all Depedencies/Library

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.7 MB 3.0 MB/s eta 0:00:35
   ---------------------------------------- 1.0/101.7 MB 12.2 MB/s eta 0:00:09
   - -------------------------------------- 3.5/101.7 MB 27.9 MB/s eta 0:00:04
   -- ------------------------------------- 6.3/101.7 MB 36.7 MB/s eta 0:00:03
   --- ------------------------------------ 7.8/101.7 MB 35.5 MB/s eta 0:00:03
   --- ------------------------------------ 9.0/101.7 MB 38.4 MB/s eta 0:00:03
   --- ------------------------------------ 9.0/101.7 MB 28.9 MB/s eta 0:00:04
   --- ------------------------------------ 9.2/101.7 MB 25.5 MB/s eta 0:00:04
   --- ------------------------------------ 9.5/101.7 MB 26.4 MB/s eta 0:00:04
   --- ------------------------------------ 9.8/101.7 MB 21.7 MB/s eta 0:00:05
   ---- ------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.


In [2]:
pip install pandasql


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26798 sha256=bfad48d3bf1144d404b5705d31a4f536f47af13565bd58ebb8e57b06f3ab806a
  Stored in directory: c:\users\12098\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


#### Import the Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import os
import datetime
import pandasql as psql
import dateutil.relativedelta
from glob import iglob # Global directory
from datetime import date
from itertools import product
from IPython.display import display, HTML # Display image on jupyter notebook
from datetime import datetime
from typing import List, Tuple, Iterable
from tqdm import tqdm # Show data processing bar
import re
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
 
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 70)

#### Data Preprocessing Function Details

1- Read_files_in_data_directory (A function that reads all .csv and .xlsx files in Data directory and returns a dictionary of DataFrames)

2- Check dataframes (Ensure all dataframes in dictionary)

3- add_year_month_and_filter_oneoff (McK_Oneoff take required columns and change column name DateTime(Year_Month_Obs)

4- transform_credit_report_to_time_series (The BSI_Creditreporting dataset, Which contains three years of horizontal data for each borrower, Needs to be converted to three years of vertical data of each borrower.)

5- rename_and_transform_logins_columns (Rename the column name & typecasting datatypes in MLW_Login Data, Extract Year_Month data)

6- count_monthly_logins (Use Year_Month Column Data, A function provide a output monthly login count each borrower)

7- add_fips_to_training_oneoff (Mapping the PropertyZipCode(McK_Oneoff) & ZIP(Add_Fips_Mapping))

8- clean_domographic (Remove the unnecessary columns)

In [14]:
# a function that reads all .csv and .xlsx files in Data directory and returns a dictionary of DataFrames 
# with filenames as keys and loaded datafrmaes as values

def read_files_in_data_directory(directory_path, is_BSI_data=True):
    # Ensure the directory path is valid
    if not os.path.isdir(directory_path):
        raise ValueError("Invalid directory path")

    # Initialize an empty dictionary to store DataFrames
    dataframes = {}
    specific_month = ''
    
    # Loop through files in the directory
    for filename in tqdm(os.listdir(directory_path), desc ="Loading Data Progress"):
        file_path = os.path.join(directory_path, filename)

        # Check if the file is a CSV or Excel file
        if filename.lower().endswith(('.csv', '.xlsx')):
            # Read the file into a DataFrame
            if filename.lower().endswith('.csv'):
                df = pd.read_csv(file_path, low_memory=False) # , error_bad_lines = False
            elif filename.lower().endswith('.xlsx'):
                df = pd.read_excel(file_path)
            
            # if 'oneoff' in filename.lower():
            if any(key_word in filename.lower() for key_word in ['one', 'off']):
                raw_data_info = filename[:-4]
                dataframe_name = 'McK_Oneoff'
                pattern = '|'.join(map(re.escape, ['_', '.', '-']))
                # Use re.split to split the string based on the delimiters
                result = re.split(pattern, filename)
                specific_month = '_'.join(result[-3:-1]).upper() + '_'
            elif 'credit' in filename.lower():
                dataframe_name = 'BSI_CREDITREPORTING'
            elif 'mlw' in filename.lower():
                dataframe_name = 'Logins'
            
            elif 'interest' in filename.lower():
                dataframe_name = 'Avg_interest_rate'
            elif 'demographic' in filename.lower():
                dataframe_name = 'Demographic'
            elif 'zip' in filename.lower():
                dataframe_name = 'Zip_fips_lookup'
            elif 'hpi' or 'house' in filename.lower():
                dataframe_name = 'Purchase_index'
            else:
                dataframe_name = os.path.splitext(filename)[0]

            print(f"{dataframe_name} is loaded: {df.shape}")
            print()
            # Add the DataFrame to the dictionary with the filename as the key
            dataframes[dataframe_name] = df
            
    if is_bsi_data:
        bsi_or_public = 'BSI'
    else:
        bsi_or_public = 'Public'
    print(f">>>>>> Loading {bsi_or_public} Data Completed <<<<<<<")
    return dataframes, specific_month, raw_data_info if is_bsi_data else dataframes



In [4]:
def check_dataframes(BSI_dataframes, public_dataframes, needed_dataframes = ['McK_Oneoff',
                                                                            'BSI_CREDITREPORTING',
                                                                            'Logins',
                                                                            'Avg_interest_rate',
                                                                            'Zip_fips_lookup',
                                                                            'Purchase_index',
                                                                            'Demographic']):
    
    combined_dataframe_dictionaries = {**BSI_dataframes,**public_dataframes}
    #  Check if all the needed dataframes exist
    try:
        for df_name in needed_dataframes:
            if df_name not in combined_dataframe_dictionaries.keys():
                raise NameError(f"{df_name} data not found!")
        print(">>>>>> All needed dataframes found <<<<<<")
    except NameError as e:
        print(f"Error: {e}")
        raise  # Raise the exception to halt the program

In [5]:
def add_year_month_and_filter_oneoff(McK_Oneoff, cols=None):
    """
    

    Parameters:
    - McK_Oneoff (pd.DataFrame): The input DataFrame containing loan data.
    - cols (list, optional): A list of column names to be selected in the final DataFrame.
                             If not provided, default columns will be selected.

    Returns:
    - pd.DataFrame: A DataFrame containing the latest occurrence of each loan with selected columns.

    Steps:
    1. Converts the 'DataDate' column to datetime format.
    2. Identifies the index of the last occurrence for each 'LoanNumber' based on 'DataDate'.
    3. Filters the DataFrame to include only the rows with the identified indices.
    4. Adds a new column 'Year_Month_Obs' formatted as '%Y-%m'.
    5. Resets the DataFrame index.
    6. Selects and retains specific columns from the DataFrame.

    Note:
    The function modifies the input DataFrame in-place and prints a message indicating the operation.
    """

    McK_Oneoff['DataDate']=pd.to_datetime(McK_Oneoff['DataDate'],format='mixed')
    # last_occurrence_index = McK_Oneoff.groupby('LoanNumber')['DataDate'].idxmax()
    # McK_Oneoff = McK_Oneoff[McK_Oneoff.index.isin(last_occurrence_index)].reset_index(drop=True)
    # clean and format
    McK_Oneoff['Year_Month_Obs']=McK_Oneoff['DataDate'].dt.strftime('%Y-%m')
    McK_Oneoff=McK_Oneoff.reset_index(drop=True)

    print(">>>>> Year_Month_Obs column added to McK_Oneoff  <<<<<")

    # select columns
    if not cols:
        cols=[
            'LoanNumber',
            'LoanType',
            'PrincipalBalance',
            'NextDueDate',
            'InterestPaidToDate',
            'DateLastPayment',
            'InterestRate',
            'OriginalInterestRate',
            'PropertyCityName',
            'PropertyState',
            'PropertyZipCode',
            'MailingCity',
            'MailingState',
            'MailingZip',
            'FICO_Score',
            'FicoDate',
            'BankruptcyFlag',
            'LossMitigationStatus',
            'ForeclosureStatus',
            'DLQ_Status',
            'NoteDate',
            'ServicingDate',
            'TransferDate',
            'PiPmt',
            'EscrowPayment',
            'EscrowBalance',
            'ReportToCreditBureau',
            'AppraisalDate',
            'AppraisedAmount',
            'Original_Ltv_Calc',
            'CurrentLTV',
            'PriorServicerName',
            'InvestorOwner',
            'AcquiredPortfolioName',
            'PrimaryBorrowerOptOut',
            'SecondaryBorrowerOptOut',
            'TermOfLoan',
            'RemainingTerm',
            'FIXED ARM',
            'SaleAmount',
            'OriginalBalance',
            'RawLeadType',
            'ForbearanceFlag',
            'Year_Month_Obs'
        ]

    McK_Oneoff = McK_Oneoff[cols]
    print(">>>>> McK_Oneoff filtered for the needed columns <<<<<")

    return  McK_Oneoff 


In [6]:
def transform_credit_report_to_time_series(BSI_CreditReporting, Year_Month_Obs=None):
    print("After implementing CB_RPTD_STA, CB_RPTG_STA and CLOSE_CD_DB000 check")
    filtered_df = BSI_CreditReporting.copy()
    print(filtered_df.shape)
    if not Year_Month_Obs:
        Year_Month_Obs=[
            '2023-01',
            '2023-02',
            '2023-03',
            '2023-04',
            '2023-05',
            '2023-06',
            '2023-07',
            '2023-08',
            '2023-09',
            '2023-10',
            '2023-11',
            '2023-12',
            '2022-01',
            '2022-02',
            '2022-03',
            '2022-04',
            '2022-05',
            '2022-06',
            '2022-07',
            '2022-08',
            '2022-09',
            '2022-10',
            '2022-11',
            '2022-12',
            '2021-01',
            '2021-02',
            '2021-03',
            '2021-04',
            '2021-05',
            '2021-06',
            '2021-07',
            '2021-08',
            '2021-09',
            '2021-10',
            '2021-11',
            '2021-12',
        ]
 
    All_LoanNumber=[]
    All_Year_Month_Obs=[]
    All_History=[]
    for loan in filtered_df['ACCT_NBR']:
        History=filtered_df[filtered_df['ACCT_NBR']==loan].iloc[:,-37:-1]
        LoanNumber=[loan]*36
        x1=LoanNumber
        x2=Year_Month_Obs
        x3=History.iloc[0,:].to_list()
        All_LoanNumber.extend(x1)
        All_Year_Month_Obs.extend(x2)
        All_History.extend(x3)
 
    data = {'LoanNumber': All_LoanNumber,
            'Year_Month_Obs': All_Year_Month_Obs,
            'CREDIT_BUREAU_PAYMENT_HISTORY':All_History}
    BSI_CreditReporting = pd.DataFrame()
    BSI_CreditReporting = pd.DataFrame(data)
    print('>>>>> Credit Report Transformed to Timeseries <<<<<<')
    return BSI_CreditReporting
 
#Function to automate 2 month lag data
#We are picking the current year month current from Training file and filter data to credit reporting file
# from datetime import datetime as dt
# from dateutil.relativedelta import relativedelta
 
# def pick_1yr_data(source_df):
#     actual_date = source_df.Year_Month_Obs.unique()[0]
#     actual_date_formatted = dt.strptime(actual_date,'%Y-%m')
#     start_date = actual_date_formatted + relativedelta(months = -13)
#     end_date = actual_date_formatted + relativedelta(months = -2)
#     return start_date, end_date
 
# #If CBPH data belongs to category ('B','D','.',0), assign 0 else 1
# def Credit_feature_enhancement(CBPH, YMO,start_date, end_date):
#     ymo = dt.strptime(YMO,'%Y-%m')
#     if ymo >= start_date and ymo <= end_date: 
#         if CBPH in ('B','D','.',0):
#             return 0
#         else:
#             return 1
#     else:
#         return 0

In [7]:
def rename_and_transform_logins_columns(logins):
    """
    Rename and transform columns in a DataFrame representing login data.

    Args:
        logins (pd.DataFrame): A DataFrame containing login data with columns 'LoanNumber' and 'Date'.

    Returns:
        pd.DataFrame: A DataFrame with columns renamed and additional columns added:
            - 'LoanNumberPlain': Renamed from 'LoanNumber'.
            - 'LoginDate': Converted to a datetime format from the original 'Date'.
            - 'Login_Year_Month': New column derived from 'LoginDate' with the format 'YYYY-MM'.

    Note:
        This function modifies the input DataFrame in place.
    """
    logins.rename(columns={'LoanNumber': 'LoanNumberPlain',
                           'Date': 'LoginDate'}, inplace=True)
    
    logins['LoginDate']=pd.to_datetime(logins['LoginDate'],format = 'mixed')
    logins['Login_Year_Month']=logins['LoginDate'].dt.strftime('%Y-%m')
    logins['LoanNumberPlain']=logins['LoanNumberPlain'].astype(int)
    print('>>>>> Logins columns renamed and transformed <<<<<<')
    return logins
    

In [8]:
def count_monthly_logins(Logins):
    """
    Count the number of logins per month for each LoanNumberPlain.

    This function uses the variable 'Logins' to generate a DataFrame
    containing the counts of logins for each LoanNumberPlain in each Login_Year_Month.

    Returns:
    DataFrame: A DataFrame containing the columns 'LoanNumberPlain',
               'Login_Year_Month', and 'Login_Count' representing the monthly
               login counts.

    Note:
    - The 'Logins' variable is assumed to be available for SQL queries.
    - The generated DataFrame includes counts of logins grouped by
      LoanNumberPlain and Login_Year_Month.
    - The function also prints a message indicating that the monthly login count
      has been generated.
    """
    Monthly_Login_Count=sqldf.run('''
                                SELECT LoanNumberPlain, Login_Year_Month, count(Login_Year_Month) as Login_Count
                                FROM Logins
                                GROUP BY LoanNumberPlain,  Login_Year_Month
                                '''
                                )
    print('>>>>> Monthly Login Count Generated <<<<<<')
    return Monthly_Login_Count

In [9]:
def add_fips_to_training_oneoff(training_oneoff, Zip_fips_lookup): # Take two argument Mck_Oneoff & add_fips
    training_oneoff['FIPS'] = np.nan # Mck_Oneoff dataset create a column name FIPS fills NaN.
    for i in range(training_oneoff.shape[0]): # Iterate the mck_oneoff data from row 0 to till end. use of shape[0].
    # for i in range(30):
        zip = int(training_oneoff.loc[i,'PropertyZipCode'][-5:]) # Taking the last character propertyzip column of the current row in mck_oneoff, convert int
        row = Zip_fips_lookup.index[Zip_fips_lookup['ZIP'] == zip] # Find the index(row number) in the Zips_fips dataframe where the ZIP column matches the extracted zip code.
        if len(row)==0: # Check the zip code ==0 then pass the else condition, if true then store the fips code in fips column
            pass
        else:
            row = row[0]
            fips = Zip_fips_lookup.loc[row,'STCOUNTYFP'] 
            training_oneoff.loc[i,'FIPS'] = fips
    print('>>>>> FIPS Added to McK_Oneoff <<<<<<')
    return training_oneoff

In [10]:
def clean_domographic(Demographic, keep_only_needed_column=False):
    Demographic = Demographic.loc[:, ~Demographic.columns.str.contains('^Unnamed')]
    columns=[
    'code',
    'geography',
    'Percent_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelors_degree_or_higher',
    'Percent_VEHICLES_AVAILABLE_Occupied_housing_units_No_vehicles_available',
    'Percent_VEHICLES_AVAILABLE_Occupied_housing_units_1_vehicle_available',
    'Percent_VEHICLES_AVAILABLE_Occupied_housing_units_3_or_more_vehicles_available',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Agriculture_forestry_fishing_and_hunting_and_mining',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Construction',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Manufacturing',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Wholesale_trade',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Retail_trade',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Transportation_and_warehousing_and_utilities',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Information',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Finance_and_insurance_and_real_estate_and_rental_and_leasing',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Professional_scientific_and_management_and_administrative_and_waste_management_services',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Educational_services_and_health_care_and_social_assistance',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Arts_entertainment_and_recreation_and_accommodation_and_food_services',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Other_services_except_public_administration',
    'Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Public_administration',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Food_Stamp_over_SNAP_benefits_in_the_past_12_months',
    'Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Median_household_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Mean_household_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings_Mean_earnings_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security_Mean_Social_Security_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income_Mean_retirement_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income_Mean_Supplemental_Security_Income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income_Mean_cash_public_assistance_income_dollars',
    'Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_private_health_insurance',
    'Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_public_coverage',
    'Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_No_health_insurance_coverage',
    'Estimate_EMPLOYMENT_STATUS_Population_16_years_and_over',
    'Percent_EMPLOYMENT_STATUS_Population_16_years_and_over_In_labor_force',
    'Percent_EMPLOYMENT_STATUS_Civilian_labor_force_Unemployment_Rate',
    'Percent_SELECTED_MONTHLY_OWNER_COSTS_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_SMOCAPI_Housing_units_with_a_mortgage_excluding_units_where_SMOCAPI_cannot_be_computed_35_0_percent_or_more',
    'Percent_SELECTED_MONTHLY_OWNER_COSTS_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_SMOCAPI_Housing_unit_without_a_mortgage_excluding_units_where_SMOCAPI_cannot_be_computed_35_0_percent_or_more',
    'Percent_GROSS_RENT_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_GRAPI_Occupied_units_paying_rent_excluding_units_where_GRAPI_cannot_be_computed_35_0_percent_or_more',
    'Percent_HOUSING_TENURE_Occupied_housing_units_Owner_occupied',
    'Percent_HOUSING_TENURE_Occupied_housing_units_Renter_occupied',
    'Percent_HOUSING_OCCUPANCY_Total_housing_units_Vacant_housing_units',
    'GINI_index',
    'Estimate_Median_gross_rent_as_a_percentage_of_household_income',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Median_household_income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Mean_household_income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings_Mean_earnings_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security_Mean_Social_Security_income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income_Mean_retirement_income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income_Mean_Supplemental_Security_Income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income_Mean_cash_public_assistance_income_dollars_1',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Families_Median_family_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Families_Mean_family_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Per_capita_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households_Median_nonfamily_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households_Mean_nonfamily_income_dollars',
    'Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Median_earnings_for_workers_dollars'
    ]
    Demographic.columns=columns

    
    return Demographic

In [11]:
# def save_data(data, path, file_name, save_sample=False, display_data=False):
#     data.to_csv(path + file_name + '.csv', index=False)
    
#     if save_sample:
#         data_head=data.sample(frac=0.01)
#         save_data(data_head, path, file_name+'_head', save_sample=False, display_data=False)
#     if display_data:
#         display(data.head())
#     print(f'>>>>> {file_name} Saved <<<<<<')


In [12]:
# def create_folders(new_folder_path, new_folder_name):
#     try:
#         # Create the directory
#         os.makedirs(new_folder_path)
#         print(f"Directory '{new_folder_name}' _created.")
#     except FileExistsError:
#         print(f"Directory '{new_folder_name}' already exists.")


#     # Create subfolders inside the Output folder
#     subfolders = ['Binning_ready_data', 'WOE_data', 'Loan_numbers', 'Predictions']
#     for subfolder in subfolders:
#         subfolder_path = os.path.join(new_folder_path, subfolder)
#         os.makedirs(subfolder_path)
#         print(f"Subfolder '{new_folder_name}/{subfolder}' created.")

In [13]:
# Update the root directory & reading the dataframes

root_directory = 'Incedo Transition Folder/1-Data Source/Training Data'
input_path = os.path.join(root_directory,'Input_Data')
BSI_data_path = os.path.join(input_path, 'BSI_data/')
public_data_path = os.path.join(input_path, 'Public_data/')


BSI_dataframes, specific_month, raw_data_info = read_files_in_data_directory(BSI_data_path) # read all BSI dataframes
 
globals().update(BSI_dataframes) # this line creates variables from dataframes keys with the same name
print('Month of Observation:', specific_month)
BSI_dataframe_names = list(BSI_dataframes.keys())
# del BSI_dataframes
print('Extracted specific month:', specific_month)
 
public_dataframes, _, _ = read_files_in_data_directory(public_data_path, False) # read all public dataframes
print(public_dataframes.keys())
globals().update(public_dataframes) # this line creates variables from dataframes keys with the same name
public_dataframe_names = list(public_dataframes.keys())
 
check_dataframes(BSI_dataframes, public_dataframes)
 
McK_Oneoff_1yr = add_year_month_and_filter_oneoff(BSI_dataframes['McK_Oneoff'])
 
print(McK_Oneoff_1yr.shape)
display(McK_Oneoff_1yr.head())

ValueError: Invalid directory path

In [15]:
# Model Training 1 yr data from Sep-22 to Aug-23

McK_Oneoff_1yr['Year_Month_Obs'] = pd.to_datetime(McK_Oneoff_1yr['Year_Month_Obs'])
print(McK_Oneoff_1yr.shape)
 
import functools
def disjunction_and(*conditions):
    return functools.reduce(np.logical_and, conditions)
 
c_1 = McK_Oneoff_1yr['Year_Month_Obs']>'2022-08' # Sep-2022 to Start
c_2 =McK_Oneoff_1yr['Year_Month_Obs']<='2023-08' # Aug-2023 to End
 
McK_Oneoff = McK_Oneoff_1yr[disjunction_and(c_1,c_2)].reset_index(drop=True)
print(McK_Oneoff.shape)
print(McK_Oneoff['Year_Month_Obs'].value_counts())

NameError: name 'McK_Oneoff_1yr' is not defined

#### Implementing & Transformation Credit Bureau Payment History Data

In [15]:
# Sort the dataframe mck_oneoff based on the values in the column "Year_month_obs" in ascending order.
# The groups the sorted dataframe by the column "LoanNumber" and select the the last row for each group. it effectly keeps only the most recent entry for each loan number.
# Reset index dataframe after grouping operation

McK_Oneoff = McK_Oneoff.sort_values(['Year_Month_Obs']).groupby(['LoanNumber']).last().reset_index()
print(McK_Oneoff.shape)
McK_Oneoff.head(2)

(164246, 44)


,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.750,2.750,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.00,0.00,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08-01
1,1235,CONVENTIONAL,53245.69,2023-08-01,2023-07-01,2023-07-28,2.875,2.875,ASTON,PA,19014,ASTON,PA,19014,608.0,2023-01-31,None,,None,1-29,2013-03-15,2013-04-05,2013-04-09,1002.92,696.66,19.31,Y,2013-02-01,232000.0,63.15,22.95,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,0.0,146500.0,Allied,None,2023-08-01


Taking 1 year data from Credit Bureau Payment based on Year month obs
 
Handling credit bureau history feature This is on the basis of last reported date column

In [16]:
# Filter the data from ['CBR_LAST_REPORT_DATE']>'2022-01']
# Take the data based on CBR_LAST_REPORT_DATE and Lag 2 month

yes_reportclosedate = BSI_CREDITREPORTING[BSI_CREDITREPORTING['CBR_LAST_REPORT_DATE']>'2022-01'].reset_index()
yes_reportclosedate['start_date'] = yes_reportclosedate.CBR_LAST_REPORT_DATE.apply(lambda x: dt.strptime(dt.strftime(x + relativedelta(months = -13),'%Y-%m'), '%Y-%m'))
yes_reportclosedate['end_date'] = yes_reportclosedate.CBR_LAST_REPORT_DATE.apply(lambda x: dt.strptime(dt.strftime(x + relativedelta(months = -2),'%Y-%m'), '%Y-%m'))
yes_reportclosedate.head()

,index,CLOSE_CD_DB000,ACCT_NBR,PRIM_BWRS_NM_DB001,CB_RPTG_STA,CBR_LAST_REPORT_DATE,CBR_RPTD_OPEND_DATE,CBR_RPTD_CLOSE_DATE,CBR_RPTD_LAST_PAYMENT_DATE,CB_RPTD_STA,CBR_COMMENT_CD,CB_RPT_CMNTS,CB_RPT_PMT_RT,CBR_RPTD_HIGH_AMOUNT,CBR_RPTD_PAYMENT_AMOUNT,CBR_RPTD_ACTUAL_PAYMENT_AMOUNT,CBR_RPTD_CURRENT_BALANCE,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_JANUARY,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_FEBRUARY,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_MARCH,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_APRIL,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_MAY,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_JUNE,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_JULY,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_AUGUST,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_SEP,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_OCTOBER,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_NOVEMBER,CREDIT_BUREAU_PAYMENT_HISTORY_THIS_YEAR_DECEMBER,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_JANUARY,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_FEBRUARY,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_MARCH,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_APRIL,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_MAY,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_JUNE,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_JULY,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_AUGUST,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_SEP,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_OCTOBER,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_NOVEMBER,CREDIT_BUREAU_PAYMENT_HISTORY_LAST_YEAR_DECEMBER,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_JAN,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_FEB,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_MARCH,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_APRIL,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_MAY,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_JUNE,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_JULY,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_AUGUST,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_SEP,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_OCT,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_NOV,CREDIT_BUREAU_PAYMENT_HISTORY_2_YEARS_AGO_DEC,DataDate,start_date,end_date
0,6,1,791,THADDEUS LUKASZEWICZ,Y,2023-10-10,2013-03-29,NaT,2023-09-01,11,,,,210000,1425,1425,68211,0,0,0,0,0,0,0,0,.,.,.,.,0,0,0,0,0,0,0,0,0,0,0,0,.,.,.,.,.,.,.,.,0,0,0,0,2023-10-31,2022-09-01,2023-08-01
1,7,1,1235,HEATHER DIIORIO,Y,2023-10-10,2013-03-15,NaT,2023-09-22,11,,,,146500,1699,3399,53247,0,0,0,1,0,0,0,1,.,.,.,.,0,0,0,0,1,1,0,0,0,1,1,0,.,.,.,.,.,.,.,.,0,0,0,0,2023-10-31,2022-09-01,2023-08-01
2,8,1,1262,LEONARD LEIB,Y,2023-10-10,2013-05-15,NaT,2023-09-01,11,,,,412500,3246,3346,136136,0,0,0,0,0,0,0,0,.,.,.,.,0,0,0,0,0,0,0,0,0,0,0,0,.,.,.,.,.,.,.,.,0,0,0,0,2023-10-31,2022-09-01,2023-08-01
3,10,1,1571,BETH A SCHULTHEIS,Y,2023-10-10,2013-05-03,NaT,2023-09-04,11,,,,100500,1042,1042,37068,0,0,0,0,0,0,0,0,.,.,.,.,0,0,0,0,0,0,0,0,0,0,0,0,.,.,.,.,.,.,.,.,0,0,0,0,2023-10-31,2022-09-01,2023-08-01
4,11,1,1596,WILLIAM J HODGE,Y,2023-10-10,2013-04-22,NaT,2023-08-01,11,,,,109100,1605,0,37174,0,0,0,0,0,0,0,0,.,.,.,.,0,0,0,0,0,0,0,0,0,0,0,0,.,.,.,.,.,.,.,.,0,0,0,0,2023-10-31,2022-09-01,2023-08-01


In [17]:
# Transform vertically all loan number with Year_Month_Obs with merging

BSI_CreditReporting_yesclosedate = transform_credit_report_to_time_series(yes_reportclosedate)
BSI_CreditReporting_yesclosedate_1 = BSI_CreditReporting_yesclosedate.merge(yes_reportclosedate[['ACCT_NBR','start_date','end_date']],how='left',left_on='LoanNumber',right_on='ACCT_NBR')
BSI_CreditReporting_yesclosedate_1.head(2)

After implementing CB_RPTD_STA, CB_RPTG_STA and CLOSE_CD_DB000 check
(174776, 56)
>>>>> Credit Report Transformed to Timeseries <<<<<<


,LoanNumber,Year_Month_Obs,CREDIT_BUREAU_PAYMENT_HISTORY,ACCT_NBR,start_date,end_date
0,791,2023-01,0,791,2022-09-01,2023-08-01
1,791,2023-02,0,791,2022-09-01,2023-08-01


In [18]:
BSI_CreditReporting_yesclosedate_1['CREDIT_BUREAU_PAYMENT_HISTORY'] = np.where(BSI_CreditReporting_yesclosedate_1['CREDIT_BUREAU_PAYMENT_HISTORY'].isin(['B','D','.',0,1,2,3,4,5,6,'H']),BSI_CreditReporting_yesclosedate_1['CREDIT_BUREAU_PAYMENT_HISTORY'],0)
BSI_CreditReporting_yesclosedate_1

,LoanNumber,Year_Month_Obs,CREDIT_BUREAU_PAYMENT_HISTORY,ACCT_NBR,start_date,end_date
0,791,2023-01,0,791,2022-09-01,2023-08-01
1,791,2023-02,0,791,2022-09-01,2023-08-01
2,791,2023-03,0,791,2022-09-01,2023-08-01
3,791,2023-04,0,791,2022-09-01,2023-08-01
4,791,2023-05,0,791,2022-09-01,2023-08-01
...,...,...,...,...,...,...
6291931,9119021467,2021-08,0,9119021467,2022-09-01,2023-08-01
6291932,9119021467,2021-09,0,9119021467,2022-09-01,2023-08-01
6291933,9119021467,2021-10,0,9119021467,2022-09-01,2023-08-01
6291934,9119021467,2021-11,0,9119021467,2022-09-01,2023-08-01


In [19]:
BSI_CreditReporting_yesclosedate_1['ymo'] = BSI_CreditReporting_yesclosedate_1['Year_Month_Obs'].apply(lambda x: dt.strptime(x, '%Y-%m'))
# Filtering data on the basis of start and end date comparing with ymo
BSI_CreditReporting_yesclosedate_1_ = BSI_CreditReporting_yesclosedate_1[BSI_CreditReporting_yesclosedate_1['ymo'] >= BSI_CreditReporting_yesclosedate_1['start_date']]
BSI_CreditReporting_yesclosedate_1_ = BSI_CreditReporting_yesclosedate_1_[BSI_CreditReporting_yesclosedate_1_['ymo'] <=BSI_CreditReporting_yesclosedate_1_['end_date']]
 
# Removing timestamp from CREDIT_BUREAU_PAYMENT_HISTORY feature
BSI_CreditReporting_yesclosedate_1_['CREDIT_BUREAU_PAYMENT_HISTORY'] = np.where(BSI_CreditReporting_yesclosedate_1_['CREDIT_BUREAU_PAYMENT_HISTORY'].isin(['B','D','.',0,1,2,3,4,5,6,'H']),BSI_CreditReporting_yesclosedate_1_['CREDIT_BUREAU_PAYMENT_HISTORY'],0)
 
# Convert 0 if CBH belongs to categories like (B, D, 0, .) else 1
BSI_CreditReporting_yesclosedate_1_['CBH'] = np.where(BSI_CreditReporting_yesclosedate_1_['CREDIT_BUREAU_PAYMENT_HISTORY'].isin(['B','D','.',0]),0,1)
 
print(BSI_CreditReporting_yesclosedate_1_.shape)
BSI_CreditReporting_yesclosedate_1_.head(2)

(2095761, 8)


,LoanNumber,Year_Month_Obs,CREDIT_BUREAU_PAYMENT_HISTORY,ACCT_NBR,start_date,end_date,ymo,CBH
0,791,2023-01,0,791,2022-09-01,2023-08-01,2023-01-01,0
1,791,2023-02,0,791,2022-09-01,2023-08-01,2023-02-01,0


In [20]:
BSI_CreditReporting_yesclosedate_1_['LoanNumber'].value_counts()

LoanNumber
791           12
1401578744    12
1401578443    12
1401578516    12
1401578519    12
              ..
1201453355    11
1218464282    11
1401206499    11
670163213     11
1201440629    11
Name: count, Length: 174776, dtype: int64

In [21]:
BSI_CreditReporting_yesclosedate_1_[BSI_CreditReporting_yesclosedate_1_['LoanNumber']==1218464282]

,LoanNumber,Year_Month_Obs,CREDIT_BUREAU_PAYMENT_HISTORY,ACCT_NBR,start_date,end_date,ymo,CBH
1213440,1218464282,2021-01,.,1218464282,2020-12-01,2021-11-01,2021-01-01,0
1213441,1218464282,2021-02,.,1218464282,2020-12-01,2021-11-01,2021-02-01,0
1213442,1218464282,2021-03,.,1218464282,2020-12-01,2021-11-01,2021-03-01,0
1213443,1218464282,2021-04,.,1218464282,2020-12-01,2021-11-01,2021-04-01,0
1213444,1218464282,2021-05,.,1218464282,2020-12-01,2021-11-01,2021-05-01,0
1213445,1218464282,2021-06,.,1218464282,2020-12-01,2021-11-01,2021-06-01,0
1213446,1218464282,2021-07,.,1218464282,2020-12-01,2021-11-01,2021-07-01,0
1213447,1218464282,2021-08,.,1218464282,2020-12-01,2021-11-01,2021-08-01,0
1213448,1218464282,2021-09,.,1218464282,2020-12-01,2021-11-01,2021-09-01,0
1213449,1218464282,2021-10,B,1218464282,2020-12-01,2021-11-01,2021-10-01,0


In [22]:
BSI_CreditReporting_yesclosedate_final = BSI_CreditReporting_yesclosedate_1_[['LoanNumber','CBH']]
BSI_CreditReporting_yesclosedate_final = BSI_CreditReporting_yesclosedate_final.groupby('LoanNumber').sum('CBH')
#BSI_CreditReporting_yesclosedate_final.rename(columns = {'dummy':'CREDIT_BUREAU_PAYMENT_HISTORY'}, inplace=True)
BSI_CreditReporting_yesclosedate_final = BSI_CreditReporting_yesclosedate_final.reset_index()
BSI_CreditReporting_yesclosedate_final.head()

,LoanNumber,CBH
0,791,0
1,1235,3
2,1262,0
3,1571,0
4,1596,0


In [23]:
print(McK_Oneoff.shape)
print(BSI_CreditReporting_yesclosedate_final.shape)
McK_Oneoff_CBH = McK_Oneoff.merge(BSI_CreditReporting_yesclosedate_final,how='left',on = 'LoanNumber')
print(McK_Oneoff_CBH.shape)
McK_Oneoff_CBH.head()

(164246, 44)
(174776, 2)
(164246, 45)


,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs,CBH
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.750,2.750,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.00,0.00,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08-01,0.0
1,1235,CONVENTIONAL,53245.69,2023-08-01,2023-07-01,2023-07-28,2.875,2.875,ASTON,PA,19014,ASTON,PA,19014,608.0,2023-01-31,None,,None,1-29,2013-03-15,2013-04-05,2013-04-09,1002.92,696.66,19.31,Y,2013-02-01,232000.0,63.15,22.95,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,0.0,146500.0,Allied,None,2023-08-01,3.0
2,1262,CONVENTIONAL,138728.50,2023-09-01,2023-08-01,2023-08-01,2.875,2.875,PHILADELPHIA,PA,19106,PHILADELPHIA,PA,19106,758.0,2023-01-31,None,,None,CURRENT,2013-05-15,2013-06-14,2013-06-21,2823.92,422.27,2955.93,Y,2013-03-01,550000.0,75.00,25.22,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,58,FIXED,550000.0,412500.0,Allied,None,2023-08-01,0.0
3,1378,CONVENTIONAL,122249.50,2013-07-01,2013-06-01,2013-06-07,4.875,4.875,LINDENWOLD,NJ,08021,LINDENWOLD,NJ,08021,503.0,2016-10-20,C,C,A,REO,2013-04-05,2013-04-26,2013-04-30,647.75,754.16,-56930.34,N,2013-03-01,153000.0,80.00,79.90,ALLIED MORTGAGE GROUP,None,None,0,0,360,359,FIXED,0.0,122400.0,Allied,None,2023-08-01,NaN
4,1571,CONVENTIONAL,37677.01,2023-09-01,2023-08-01,2023-08-04,3.500,3.500,UPPER DARBY,PA,19082,UPPER DARBY,PA,19082,801.0,2023-01-31,None,,None,CURRENT,2013-05-03,2013-05-24,2013-05-29,718.46,324.03,626.03,Y,2013-04-01,104000.0,96.63,36.23,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,104000.0,100500.0,Allied,None,2023-08-01,0.0


In [24]:
McK_Oneoff_CBH["CBH"].isna().sum()

1639

In [25]:
Training_Oneoff=McK_Oneoff_CBH.merge(Avg_interest_rate, how='left',left_on=['LoanNumber'],right_on=['LoanNumber'])

In [26]:
Training_Oneoff.shape

(164246, 53)

In [27]:
Training_Oneoff.head(1)

,LoanNumber,LoanType_x,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs,CBH,Unnamed: 0,LoanType_y,Year,Month,Term,FICO,LTV,Orginal_Interest_Rate
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.75,2.75,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.0,0.0,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08-01,0.0,0,CONVENTIONAL,2023,6,15,>=760,<=30,6.6275


In [28]:
Training_Oneoff.columns.tolist()

['LoanNumber',
 'LoanType_x',
 'PrincipalBalance',
 'NextDueDate',
 'InterestPaidToDate',
 'DateLastPayment',
 'InterestRate',
 'OriginalInterestRate',
 'PropertyCityName',
 'PropertyState',
 'PropertyZipCode',
 'MailingCity',
 'MailingState',
 'MailingZip',
 'FICO_Score',
 'FicoDate',
 'BankruptcyFlag',
 'LossMitigationStatus',
 'ForeclosureStatus',
 'DLQ_Status',
 'NoteDate',
 'ServicingDate',
 'TransferDate',
 'PiPmt',
 'EscrowPayment',
 'EscrowBalance',
 'ReportToCreditBureau',
 'AppraisalDate',
 'AppraisedAmount',
 'Original_Ltv_Calc',
 'CurrentLTV',
 'PriorServicerName',
 'InvestorOwner',
 'AcquiredPortfolioName',
 'PrimaryBorrowerOptOut',
 'SecondaryBorrowerOptOut',
 'TermOfLoan',
 'RemainingTerm',
 'FIXED ARM',
 'SaleAmount',
 'OriginalBalance',
 'RawLeadType',
 'ForbearanceFlag',
 'Year_Month_Obs',
 'CBH',
 'Unnamed: 0',
 'LoanType_y',
 'Year',
 'Month',
 'Term',
 'FICO',
 'LTV',
 'Orginal_Interest_Rate']

In [29]:
Training_Oneoff["LoanType_x"].isna().sum()

1

In [30]:
Training_Oneoff["LoanType_y"].isna().sum()

0

In [31]:
cols=[
 'LoanNumber',
 'LoanType_y',
#  'LoanType_x',
 'PrincipalBalance',
 'NextDueDate',
 'InterestPaidToDate',
 'DateLastPayment',
 'InterestRate',
 'OriginalInterestRate',
 'PropertyCityName',
 'PropertyState',
 'PropertyZipCode',
 'MailingCity',
 'MailingState',
 'MailingZip',
 'FICO_Score',
 'FicoDate',
 'BankruptcyFlag',
 'LossMitigationStatus',
 'ForeclosureStatus',
 'DLQ_Status',
 'NoteDate',
 'ServicingDate',
 'TransferDate',
 'PiPmt',
 'EscrowPayment',
 'EscrowBalance',
 'ReportToCreditBureau',
 'AppraisalDate',
 'AppraisedAmount',
 'Original_Ltv_Calc',
 'CurrentLTV',
 'PriorServicerName',
 'InvestorOwner',
 'AcquiredPortfolioName',
 'PrimaryBorrowerOptOut',
 'SecondaryBorrowerOptOut',
 'TermOfLoan',
 'RemainingTerm',
 'FIXED ARM',
 'SaleAmount',
 'OriginalBalance',
 'RawLeadType',
 'ForbearanceFlag',
 'Year_Month_Obs',
 'CBH',
 'Orginal_Interest_Rate',
#  'Unnamed: 0',
 'Year',
 'Month',
 'Term',
 'FICO',
 'LTV',  
]

In [32]:
Training_Oneoff=Training_Oneoff[cols]

In [33]:
Training_Oneoff.rename({'LoanType_y':'LoanType'},axis=1, inplace=True)

In [34]:
Training_Oneoff.head(1)

,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs,CBH,Orginal_Interest_Rate,Year,Month,Term,FICO,LTV
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.75,2.75,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.0,0.0,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08-01,0.0,6.6275,2023,6,15,>=760,<=30


In [35]:
Logins = rename_and_transform_logins_columns(BSI_dataframes['Logins'])
display(Logins.head())

>>>>> Logins columns renamed and transformed <<<<<<


,LoanNumberPlain,LoginDate,Login_Year_Month
0,1119,2021-02-26 13:22:54.343,2021-02
1,1119,2021-02-28 14:17:27.033,2021-02
2,1119,2021-11-30 10:54:51.877,2021-11
3,1119,2022-02-22 18:14:32.110,2022-02
4,1119,2022-06-26 13:35:29.967,2022-06


In [36]:
Monthly_Login_Count = count_monthly_logins(Logins)
display(Monthly_Login_Count)

>>>>> Monthly Login Count Generated <<<<<<


,LoanNumberPlain,Login_Year_Month,Login_Count
0,-2145945248,2022-09,1
1,-2145945248,2023-05,1
2,-2145945215,2022-08,4
3,-2145945215,2022-09,8
4,-2145945215,2022-10,5
...,...,...,...
1666652,2140630000,2023-04,1
1666653,2140630000,2023-05,1
1666654,2140630000,2023-06,1
1666655,2140630000,2023-07,2


In [37]:
McK_Oneoff.dtypes

LoanNumber                          int64
LoanType                           object
PrincipalBalance                  float64
NextDueDate                        object
InterestPaidToDate                 object
DateLastPayment                    object
InterestRate                      float64
OriginalInterestRate              float64
PropertyCityName                   object
PropertyState                      object
PropertyZipCode                    object
MailingCity                        object
MailingState                       object
MailingZip                         object
FICO_Score                        float64
FicoDate                           object
BankruptcyFlag                     object
LossMitigationStatus               object
ForeclosureStatus                  object
DLQ_Status                         object
NoteDate                           object
ServicingDate                      object
TransferDate                       object
PiPmt                             

In [38]:
Training_Oneoff["CBH"].dtypes

dtype('float64')

In [39]:
Training_Oneoff["Orginal_Interest_Rate"].dtypes

dtype('float64')

In [40]:
Monthly_Login_Count.dtypes

LoanNumberPlain      int64
Login_Year_Month    object
Login_Count          int64
dtype: object

In [41]:
import re
Monthly_Login_Count['LoanNumberPlain']=Monthly_Login_Count['LoanNumberPlain'].astype(str).str.replace(r'^-','',regex=True)

In [42]:
display(Monthly_Login_Count)

,LoanNumberPlain,Login_Year_Month,Login_Count
0,2145945248,2022-09,1
1,2145945248,2023-05,1
2,2145945215,2022-08,4
3,2145945215,2022-09,8
4,2145945215,2022-10,5
...,...,...,...
1666652,2140630000,2023-04,1
1666653,2140630000,2023-05,1
1666654,2140630000,2023-06,1
1666655,2140630000,2023-07,2


In [43]:
Column_Data=Monthly_Login_Count['LoanNumberPlain']
for value in Column_Data:
    if'-' in str(value):
        print(value)

In [44]:
Monthly_Login_Count['LoanNumberPlain']=Monthly_Login_Count["LoanNumberPlain"].astype(int)

In [45]:
Monthly_Login_Count.dtypes

LoanNumberPlain      int32
Login_Year_Month    object
Login_Count          int64
dtype: object

In [46]:
Training_Oneoff["Year_Month_Obs"].dtypes

dtype('<M8[ns]')

In [47]:
Training_Oneoff["Year_Month_Obs"]=Training_Oneoff["Year_Month_Obs"].dt.strftime("%Y-%m")

In [48]:
Training_Oneoff["Year_Month_Obs"]=Training_Oneoff["Year_Month_Obs"].astype("str")

In [49]:
Training_Oneoff["Year_Month_Obs"].head(1)

0    2023-08
Name: Year_Month_Obs, dtype: object

In [50]:
Training_Oneoff[['LoanNumber','Year_Month_Obs']].dtypes

LoanNumber         int64
Year_Month_Obs    object
dtype: object

In [51]:
Monthly_Login_Count[['LoanNumberPlain','Login_Year_Month']].dtypes

LoanNumberPlain      int32
Login_Year_Month    object
dtype: object

In [52]:
Monthly_Login_Count[Monthly_Login_Count["LoanNumberPlain"]==791]

,LoanNumberPlain,Login_Year_Month,Login_Count
155885,791,2021-12,1
155886,791,2022-01,4
155887,791,2022-03,1


In [53]:
McK_Oneoff[McK_Oneoff["LoanNumber"]==791]

,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.75,2.75,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.0,0.0,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08-01


In [54]:
Training_Oneoff_Data = Training_Oneoff.merge(Monthly_Login_Count, how='left',left_on=['LoanNumber','Year_Month_Obs'],right_on=['LoanNumberPlain','Login_Year_Month'])
display(Training_Oneoff_Data.head())

,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs,CBH,Orginal_Interest_Rate,Year,Month,Term,FICO,LTV,LoanNumberPlain,Login_Year_Month,Login_Count
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.750,2.750,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.00,0.00,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=760,<=30,NaN,NaN,NaN
1,1235,CONVENTIONAL,53245.69,2023-08-01,2023-07-01,2023-07-28,2.875,2.875,ASTON,PA,19014,ASTON,PA,19014,608.0,2023-01-31,None,,None,1-29,2013-03-15,2013-04-05,2013-04-09,1002.92,696.66,19.31,Y,2013-02-01,232000.0,63.15,22.95,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,0.0,146500.0,Allied,None,2023-08,3.0,6.6275,2023,6,15,<620,<=30,NaN,NaN,NaN
2,1262,CONVENTIONAL,138728.50,2023-09-01,2023-08-01,2023-08-01,2.875,2.875,PHILADELPHIA,PA,19106,PHILADELPHIA,PA,19106,758.0,2023-01-31,None,,None,CURRENT,2013-05-15,2013-06-14,2013-06-21,2823.92,422.27,2955.93,Y,2013-03-01,550000.0,75.00,25.22,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,58,FIXED,550000.0,412500.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=740,<=30,NaN,NaN,NaN
3,1378,CONVENTIONAL,122249.50,2013-07-01,2013-06-01,2013-06-07,4.875,4.875,LINDENWOLD,NJ,08021,LINDENWOLD,NJ,08021,503.0,2016-10-20,C,C,A,REO,2013-04-05,2013-04-26,2013-04-30,647.75,754.16,-56930.34,N,2013-03-01,153000.0,80.00,79.90,ALLIED MORTGAGE GROUP,None,None,0,0,360,359,FIXED,0.0,122400.0,Allied,None,2023-08,NaN,6.6275,2023,6,30,<620,<=80,NaN,NaN,NaN
4,1571,CONVENTIONAL,37677.01,2023-09-01,2023-08-01,2023-08-04,3.500,3.500,UPPER DARBY,PA,19082,UPPER DARBY,PA,19082,801.0,2023-01-31,None,,None,CURRENT,2013-05-03,2013-05-24,2013-05-29,718.46,324.03,626.03,Y,2013-04-01,104000.0,96.63,36.23,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,104000.0,100500.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=780,<=60,NaN,NaN,NaN


In [55]:
Training_Oneoff_Data_Record = add_fips_to_training_oneoff(Training_Oneoff_Data, public_dataframes['Zip_fips_lookup'])
display(Training_Oneoff_Data_Record.head())

>>>>> FIPS Added to McK_Oneoff <<<<<<


,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,SecondaryBorrowerOptOut,TermOfLoan,RemainingTerm,FIXED ARM,SaleAmount,OriginalBalance,RawLeadType,ForbearanceFlag,Year_Month_Obs,CBH,Orginal_Interest_Rate,Year,Month,Term,FICO,LTV,LoanNumberPlain,Login_Year_Month,Login_Count,FIPS
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.750,2.750,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.00,0.00,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,0,180,55,FIXED,0.0,210000.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=760,<=30,NaN,NaN,NaN,34009.0
1,1235,CONVENTIONAL,53245.69,2023-08-01,2023-07-01,2023-07-28,2.875,2.875,ASTON,PA,19014,ASTON,PA,19014,608.0,2023-01-31,None,,None,1-29,2013-03-15,2013-04-05,2013-04-09,1002.92,696.66,19.31,Y,2013-02-01,232000.0,63.15,22.95,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,0.0,146500.0,Allied,None,2023-08,3.0,6.6275,2023,6,15,<620,<=30,NaN,NaN,NaN,42045.0
2,1262,CONVENTIONAL,138728.50,2023-09-01,2023-08-01,2023-08-01,2.875,2.875,PHILADELPHIA,PA,19106,PHILADELPHIA,PA,19106,758.0,2023-01-31,None,,None,CURRENT,2013-05-15,2013-06-14,2013-06-21,2823.92,422.27,2955.93,Y,2013-03-01,550000.0,75.00,25.22,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,58,FIXED,550000.0,412500.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=740,<=30,NaN,NaN,NaN,42101.0
3,1378,CONVENTIONAL,122249.50,2013-07-01,2013-06-01,2013-06-07,4.875,4.875,LINDENWOLD,NJ,08021,LINDENWOLD,NJ,08021,503.0,2016-10-20,C,C,A,REO,2013-04-05,2013-04-26,2013-04-30,647.75,754.16,-56930.34,N,2013-03-01,153000.0,80.00,79.90,ALLIED MORTGAGE GROUP,None,None,0,0,360,359,FIXED,0.0,122400.0,Allied,None,2023-08,NaN,6.6275,2023,6,30,<620,<=80,NaN,NaN,NaN,34007.0
4,1571,CONVENTIONAL,37677.01,2023-09-01,2023-08-01,2023-08-04,3.500,3.500,UPPER DARBY,PA,19082,UPPER DARBY,PA,19082,801.0,2023-01-31,None,,None,CURRENT,2013-05-03,2013-05-24,2013-05-29,718.46,324.03,626.03,Y,2013-04-01,104000.0,96.63,36.23,ALLIED MORTGAGE GROUP,Balbec Fund III,ALLIED [1 - FNMA],0,0,180,57,FIXED,104000.0,100500.0,Allied,None,2023-08,0.0,6.6275,2023,6,15,>=780,<=60,NaN,NaN,NaN,42045.0


In [56]:
Training_Oneoff_Data_Record.shape

(164246, 55)

In [57]:
Training_Oneoff_Data_Record["CBH"].isna().sum()

1639

In [58]:
Demographic = clean_domographic(public_dataframes['Demographic'])
display(Demographic.head())

,code,geography,Percent_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelors_degree_or_higher,Percent_VEHICLES_AVAILABLE_Occupied_housing_units_No_vehicles_available,Percent_VEHICLES_AVAILABLE_Occupied_housing_units_1_vehicle_available,Percent_VEHICLES_AVAILABLE_Occupied_housing_units_3_or_more_vehicles_available,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Agriculture_forestry_fishing_and_hunting_and_mining,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Construction,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Manufacturing,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Wholesale_trade,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Retail_trade,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Transportation_and_warehousing_and_utilities,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Information,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Finance_and_insurance_and_real_estate_and_rental_and_leasing,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Professional_scientific_and_management_and_administrative_and_waste_management_services,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Educational_services_and_health_care_and_social_assistance,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Arts_entertainment_and_recreation_and_accommodation_and_food_services,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Other_services_except_public_administration,Percent_INDUSTRY_Civilian_employed_population_16_years_and_over_Public_administration,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Food_Stamp_over_SNAP_benefits_in_the_past_12_months,Percent_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Median_household_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Mean_household_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings_Mean_earnings_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security_Mean_Social_Security_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income_Mean_retirement_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income_Mean_Supplemental_Security_Income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income_Mean_cash_public_assistance_income_dollars,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_private_health_insurance,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_public_coverage,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_No_health_insurance_coverage,Estimate_EMPLOYMENT_STATUS_Population_16_years_and_over,Percent_EMPLOYMENT_STATUS_Population_16_years_and_over_In_labor_force,Percent_EMPLOYMENT_STATUS_Civilian_labor_force_Unemployment_Rate,Percent_SELECTED_MONTHLY_OWNER_COSTS_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_SMOCAPI_Housing_units_with

In [59]:
Training_Oneoff_Data_Record=Training_Oneoff_Data_Record.merge(Demographic, how='left',left_on=['FIPS'],right_on=['geography'])
Training_Oneoff_Data_Record=Training_Oneoff_Data_Record.merge(Purchase_index, how='left',left_on=['Year_Month_Obs','PropertyState'],right_on=['Year_Month_Index','state'])
display(Training_Oneoff_Data_Record.head())
print(Training_Oneoff_Data_Record.shape)

,LoanNumber,LoanType,PrincipalBalance,NextDueDate,InterestPaidToDate,DateLastPayment,InterestRate,OriginalInterestRate,PropertyCityName,PropertyState,PropertyZipCode,MailingCity,MailingState,MailingZip,FICO_Score,FicoDate,BankruptcyFlag,LossMitigationStatus,ForeclosureStatus,DLQ_Status,NoteDate,ServicingDate,TransferDate,PiPmt,EscrowPayment,EscrowBalance,ReportToCreditBureau,AppraisalDate,AppraisedAmount,Original_Ltv_Calc,CurrentLTV,PriorServicerName,InvestorOwner,AcquiredPortfolioName,PrimaryBorrowerOptOut,...,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_private_health_insurance,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_With_health_insurance_coverage_With_public_coverage,Percent_HEALTH_INSURANCE_COVERAGE_Civilian_noninstitutionalized_population_No_health_insurance_coverage,Estimate_EMPLOYMENT_STATUS_Population_16_years_and_over,Percent_EMPLOYMENT_STATUS_Population_16_years_and_over_In_labor_force,Percent_EMPLOYMENT_STATUS_Civilian_labor_force_Unemployment_Rate,Percent_SELECTED_MONTHLY_OWNER_COSTS_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_SMOCAPI_Housing_units_with_a_mortgage_excluding_units_where_SMOCAPI_cannot_be_computed_35_0_percent_or_more,Percent_SELECTED_MONTHLY_OWNER_COSTS_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_SMOCAPI_Housing_unit_without_a_mortgage_excluding_units_where_SMOCAPI_cannot_be_computed_35_0_percent_or_more,Percent_GROSS_RENT_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_GRAPI_Occupied_units_paying_rent_excluding_units_where_GRAPI_cannot_be_computed_35_0_percent_or_more,Percent_HOUSING_TENURE_Occupied_housing_units_Owner_occupied,Percent_HOUSING_TENURE_Occupied_housing_units_Renter_occupied,Percent_HOUSING_OCCUPANCY_Total_housing_units_Vacant_housing_units,GINI_index,Estimate_Median_gross_rent_as_a_percentage_of_household_income,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Median_household_income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Mean_household_income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_earnings_Mean_earnings_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Social_Security_Mean_Social_Security_income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_retirement_income_Mean_retirement_income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_Supplemental_Security_Income_Mean_Supplemental_Security_Income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_With_cash_public_assistance_income_Mean_cash_public_assistance_income_dollars_1,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Families_Median_family_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Families_Mean_family_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Per_capita_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households_Median_nonfamily_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Nonfamily_households_Mean_nonfamily_income_dollars,Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Median_earnings_for_workers_dollars,state,yr,qtr,index_nsa,index_sa,Year_Month_Index,Index_delta_4Qs_nsa,Index_delta_4Qs_sa
0,791,CONVENTIONAL,69477.71,2023-10-01,2023-09-01,2023-08-01,2.750,2.750,NORTH WILDWOOD,NJ,08260,SICKLERVILLE,NJ,08081,777.0,2023-01-31,None,,None,CURRENT,2013-03-29,2013-04-19,2013-04-24,1425.11,0.00,0.00,Y,2013-02-01,315000.0,66.67,22.06,ALLIED MORTGAGE GROUP,Balbec Fund V,ALLIED [2 - FNMA],0,...,78.7,40.8,4.9,81385.0,56.6,6.3,26.0,13.9,51.1,78.1,21.9,50.8,0.4850,35.3,78657.0,108949.0,100778.0,24146.0,48102.0,10694.0,3462.0,92443.0,131956.0,53482.0,47500.0,61664.0,49664.0,NaN,Na

(164246, 123)


In [60]:
Training_Oneoff_Data_Record["CBH"]

0         0.0
1         3.0
2         0.0
3         NaN
4         0.0
         ... 
164241    0.0
164242    NaN
164243    NaN
164244    0.0
164245    0.0
Name: CBH, Length: 164246, dtype: float64

In [61]:
Training_Oneoff_Data_Record["Orginal_Interest_Rate"]

0         6.6275
1         6.6275
2         6.6275
3         6.6275
4         6.6275
           ...  
164241    6.6275
164242    6.6275
164243    6.6275
164244    6.6275
164245    6.6275
Name: Orginal_Interest_Rate, Length: 164246, dtype: float64

In [62]:
Training_Oneoff_Data_Record.rename(columns = {'CBH':'CREDIT_BUREAU_PAYMENT_HISTORY'}, inplace = True)

In [63]:
Training_Oneoff_Data_Record["CREDIT_BUREAU_PAYMENT_HISTORY"].isna().sum()

1639

In [64]:
# Transformation a New Features for used the Model Training

Training_Oneoff_Data_Record['Is_MailingZip_Equal_PropZip']=(Training_Oneoff_Data_Record['PropertyZipCode'].
                                                            astype(str)==Training_Oneoff_Data_Record['MailingZip'].
                                                            astype(str).apply(lambda x: x[:6])).astype(int)

Training_Oneoff_Data_Record['Interest_Delta_Current_Orginal']=(Training_Oneoff_Data_Record['InterestRate']-
                                                               Training_Oneoff_Data_Record['OriginalInterestRate']) # This features not used in model training, just for analysis to check.

Training_Oneoff_Data_Record['Interest_Delta_Current_Market']=(Training_Oneoff_Data_Record['InterestRate']-
                                             Training_Oneoff_Data_Record['Orginal_Interest_Rate'])

Training_Oneoff_Data_Record['Ltv_Delta_Orginal_Current']=(Training_Oneoff_Data_Record['Original_Ltv_Calc']-
                                         Training_Oneoff_Data_Record['CurrentLTV'])

Training_Oneoff_Data_Record['CREDIT_BUREAU_PAYMENT_HISTORY']=Training_Oneoff_Data_Record['CREDIT_BUREAU_PAYMENT_HISTORY'].fillna(0)

In [65]:
# Shape Check

Training_Oneoff_Data_Record.shape

(164246, 127)

In [66]:
# Select the model training feature

Training_Oneoff_Data_Record=Training_Oneoff_Data_Record[["LoanNumber","LoanType","CREDIT_BUREAU_PAYMENT_HISTORY",
                                                        "PrincipalBalance","Is_MailingZip_Equal_PropZip","FICO_Score",
                                                        "CurrentLTV","Login_Count","Interest_Delta_Current_Market",
                                                         "Ltv_Delta_Orginal_Current",
                                                         "Percent_EDUCATIONAL_ATTAINMENT_Population_25_years_and_over_Bachelors_degree_or_higher",
                                                        "Estimate_INCOME_AND_BENEFITS_IN_2021_INFLATION_ADJUSTED_DOLLARS_Total_households_Mean_household_income_dollars",
                                                        "Percent_HOUSING_TENURE_Occupied_housing_units_Owner_occupied"]]

In [68]:
# Model Training Data

Training_Oneoff_Data_Record.to_csv("Incedo Transition Folder/1-Data Source/Training Data/Output_Data/BSI_One_Year_Training_Data.csv",index=False)

#### Completed Data Merging Below Files
1- MCK_ONE_OFF

2- Interest_Rate(New)

3- BSI Credit Reporting (New)

4- Census_Demographics

5- HPI

6- ZIP_SIPS

7- MLW_LOGINS

### Important Points

This code file generated the BSI_one_Year_Training_Data, THis data save in "Incedo Transition Folder/1-Data Source/Training Data/Output_Data/BSI_One_Year_Training_Data.csv"

## Completed